In [2]:
import copy

from constraint import Problem
from random import randint

from operations import addition, multiplication, division, zero
from problem_constraints import add_constraint_for_addition, add_constraint_for_division, add_constraint_for_multiplication, add_constraint_for_zero
from utils import get_solutions_for_one_variable


In [ ]:

def choose_operation(x: int, candidates: list[int]) -> str:
    """
    Choose the best operation to narrow down candidates based on current x value.
    Returns: 'addition', 'multiplication', 'division', or 'zero'
    """
    
    if len(candidates) <= 1:
        return 'addition'  # Any operation works if only one candidate
    
    # Calculate range of candidates
    min_cand = min(candidates)
    max_cand = max(candidates)
    spread = max_cand - min_cand
    
    # Check if candidates have different last digits
    last_digits = set(c % 10 for c in candidates)
    
    # Check if candidates have different tens digits
    tens_digits = set(c // 10 for c in candidates)
    
    # Check if some candidates are multiples of 10
    has_multiples_of_10 = any(c % 10 == 0 for c in candidates)
    has_non_multiples_of_10 = any(c % 10 != 0 for c in candidates)
    
    # PRIORITY 1: Use multiplication if candidates differ by last digit
    if len(last_digits) > 1:
        return 'multiplication'
    
    # PRIORITY 2: Use zero if candidates differ by tens digit OR mix of multiples/non-multiples of 10
    if len(tens_digits) > 1 or (has_multiples_of_10 and has_non_multiples_of_10):
        return 'zero'
    
    # PRIORITY 3: Use division for magnitude comparison (when spread is large)
    if spread > 20:
        # Check if candidates are on different sides of x
        smaller = [c for c in candidates if c < x]
        larger = [c for c in candidates if c > x]
        if smaller and larger:
            return 'division'
        
        # Or if we can get different quotients
        quotients = set(max(c, x) // min(c, x) if min(c, x) > 0 else 0 for c in candidates)
        if len(quotients) > 1:
            return 'division'
    
    # PRIORITY 4: Use addition for range detection
    # Check if candidates would give different results with addition
    sums = set()
    for c in candidates:
        s = x + c
        if s >= 180:
            sums.add('high')  # Random result
        elif s <= 20:
            sums.add('low')   # Random result
        else:
            sums.add(s)
    
    if len(sums) > 1:
        return 'addition'
    
    # DEFAULT: multiplication (most reliable for small differences)
    return 'multiplication'



In [4]:

# Example usage:
if __name__ == "__main__":
    x = 42
    candidates = [37, 38, 42, 47, 52]
    
    operation = choose_operation(x, candidates)
    print(f"With x={x} and candidates={candidates}")
    print(f"Best operation to use: {operation}")
    
    # More examples
    print("\n--- More examples ---")
    
    # Different last digits -> multiplication
    candidates = [33, 35, 37]
    print(f"\nCandidates with different last digits: {candidates}")
    print(f"Operation: {choose_operation(x, candidates)}")
    
    # Different tens digits -> zero
    candidates = [25, 35, 45]
    print(f"\nCandidates with different tens digits: {candidates}")
    print(f"Operation: {choose_operation(x, candidates)}")
    
    # Large spread -> division
    candidates = [10, 50, 90]
    print(f"\nCandidates with large spread: {candidates}")
    print(f"Operation: {choose_operation(x, candidates)}")
    
    # Near boundary -> addition
    x = 150
    candidates = [25, 30, 35]
    print(f"\nWith x={x} near upper boundary, candidates: {candidates}")
    print(f"Operation: {choose_operation(x, candidates)}")

With x=42 and candidates=[37, 38, 42, 47, 52]
Best operation to use: multiplication

--- More examples ---

Candidates with different last digits: [33, 35, 37]
Operation: multiplication

Candidates with different tens digits: [25, 35, 45]
Operation: zero

Candidates with large spread: [10, 50, 90]
Operation: zero

With x=150 near upper boundary, candidates: [25, 30, 35]
Operation: multiplication
